In [15]:
# File: recommendation_system_with_tags.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Load datasets
movies = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ratings.csv')
tags = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/tags.csv')

# Merge datasets
data = pd.merge(ratings, movies, on='movieId')

# Create pivot table for collaborative filtering
pivot_table = data.pivot_table(index='userId', columns='title', values='rating')

# Normalize ratings
pivot_table.fillna(0, inplace=True)
user_similarity = cosine_similarity(pivot_table)
user_similarity_df = pd.DataFrame(user_similarity, index=pivot_table.index, columns=pivot_table.index)

# Combine tags for each movie for content-based filtering
tags['tag'] = tags['tag'].fillna('').str.lower()
tags_grouped = tags.groupby('movieId')['tag'].apply(lambda x: ' '.join(x)).reset_index()

# Merge tags with movies
movies_with_tags = pd.merge(movies, tags_grouped, on='movieId', how='left').fillna('')

# Generate TF-IDF matrix for movie tags
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_with_tags['tag'])

# Compute similarity between movies based on tags
movie_similarity = cosine_similarity(tfidf_matrix)
movie_similarity_df = pd.DataFrame(movie_similarity, index=movies_with_tags['title'], columns=movies_with_tags['title'])

# Hybrid recommendation function
def hybrid_recommend_movies(user_id, movie_title=None, n=5):
    # Collaborative filtering recommendations
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:n+1]
    recommended_movies_collab = set()

    for similar_user in similar_users.index:
        movies_rated = pivot_table.loc[similar_user][pivot_table.loc[similar_user] > 0].index
        recommended_movies_collab.update(movies_rated)

    # Content-based filtering recommendations
    recommended_movies_content = []
    if movie_title and movie_title in movie_similarity_df.index:
        similar_movies = movie_similarity_df[movie_title].sort_values(ascending=False)[1:n+1]
        recommended_movies_content = similar_movies.index.tolist()

    # Combine recommendations
    hybrid_recommendations = list(recommended_movies_collab)
    if recommended_movies_content:
        hybrid_recommendations.extend(recommended_movies_content)

    return list(set(hybrid_recommendations))[:n]

# Example usage
user_id = 2
movie_title = "Toy Story (1995)"  # Optional: Pass a movie title for content-based filtering
recommended = hybrid_recommend_movies(user_id, movie_title)
print(f"Movies recommended for User {user_id} based on hybrid approach: {recommended}")


Movies recommended for User 2 based on hybrid approach: ['Harry Potter and the Prisoner of Azkaban (2004)', 'The Hunger Games (2012)', 'Mr. Nobody (2009)', 'Old Boy (2003)', 'Get Out (2017)']
